In [ ]:
#Load MNIST Dataset
#MNIST Dataset can be found here: http://yann.lecun.com/exdb/mn
import scipy
import matplotlib.pyplot as plt
import gzip
from math import sqrt
import random
import pandas as pd
#Program to load the data 
def load_mnist(filename, type, n_datapoints):
    # MNIST Images have 28*28 pixels dimension
    image_size = 28
    f = gzip.open(filename)
    if(type == 'image'):
        f.read(16)  # Skip Non-Image information
        buf = f.read(n_datapoints * image_size * image_size)
        data = np.frombuffer(buf, dtype=np.uint8).astype(np.float32)
        data = data.reshape(n_datapoints, image_size, image_size, 1)
    elif(type == 'label'):
        f.read(8) # Skip Inessential information
        buf = f.read(n_datapoints)
        data = np.frombuffer(buf, dtype=np.uint8).astype(np.int64)
        data = data.reshape(n_datapoints, 1)
    return data

# Training Dataset
train_size = 60000
test_size = 10000
# dirpath = '/content/drive/My Drive/02 MNIST Digit Recognition/'
dirpath = ''
X_train = load_mnist('train-images.gz', 'image', train_size)
train_label = load_mnist('train-labels.gz','image', train_size)
X_test = load_mnist('test-images.gz','image', test_size)
test_label = load_mnist('test-labels.gz', 'label', test_size)
#To verify whether the data is loaded properly, the image and label are randomly selected and visualized


index = random.randint(0, train_size)
print('Index: ', index)
print('Training Set: ')
print('Label:', y[index])
img = np.asarray(X[index]).squeeze()
plt.imshow(img)
plt.show()

index = random.randint(0, test_size)
print('Index: ', index)
print('Training Set: ')
print('Label:', y_test[index])
img = np.asarray(X_test[index]).squeeze()
plt.imshow(img)
plt.show()
#KNN CLASS
class KNN():
  def _init_(self,k):
    self.k=k
    print(self.k)
  def fit(self,X_train,y_train):
    self.x_train=X_train
    self.y_train=y_train
  def calculate_euclidean(self,sample1,sample2):
    distance=0.0
    for i in range(len(sample1)):
      distance+=(sample1[i]-sample2[i])**2 #Euclidean Distance = sqrt(sum i to N (x1_i – x2_i)^2)
    return sqrt(distance)
  def nearest_neighbors(self,test_sample):
    distances=[]#calculate distances from a test sample to every sample in a training set
    for i in range(len(self.x_train)):
      distances.append((self.y_train[i],self.calculate_euclidean(self.x_train[i],test_sample)))
      distances.sort(key=lambda x:x[1])#sort in ascending order, based on a distance value
    neighbors=[]
    for i in range(self.k): #get first k samples
      neighbors.append(distances[i][0])
    return neighbors
  def predict(self,test_set):
    predictions=[]
    for test_sample in test_set:
      neighbors=self.nearest_neighbors(test_sample)
      labels=[sample for sample in neighbors]
      prediction=max(labels,key=labels.count)
      predictions.append(prediction)
    return predictions

model=KNN(5) #our model
model.fit(X_train,y_train)  
def accuracy_metric(actual, predicted):
	correct = 0
	for i,j in zip(actual, predicted):
		if i == j:
		   correct += 1
	return correct / float(len(actual)) * 100.0


#CONFUSION MATRIX
def  confusion_matrix(actual,predicted):
      classes = set(actual)
      number_of_classes = len(classes)
      conf_matrix = pd.DataFrame(np.zeros((number_of_classes, number_of_classes),dtype=int),
      index=classes,
      columns=classes)
      for i, j in zip(actual,predicted):
      conf_matrix.loc[i, j] += 1
      return conf_matrix
def distance_matrix(A, B, squared=False):
    """
    Compute all pairwise distances between vectors in A and B.

    Parameters
    ----------
    A : np.array
        shape should be (M, K)
    B : np.array
        shape should be (N, K)

    Returns
    -------
    D : np.array
        A matrix D of shape (M, N).  Each entry in D i,j represnets the
        distance between row i in A and row j in B.

    """
    M = A.shape[0]
    N = B.shape[0]

    assert A.shape[1] == B.shape[1], f"The number of components for vectors in A \
        {A.shape[1]} does not match that of B {B.shape[1]}!"
        
    A_dots = (A*A).sum(axis=1).reshape((M,1))*np.ones(shape=(1,N))
    B_dots = (B*B).sum(axis=1)*np.ones(shape=(M,1))
    D_squared =  A_dots + B_dots -2*A.dot(B.T)

    if squared == False:
        zero_mask = np.less(D_squared, 0.0)
        D_squared[zero_mask] = 0.0
        return np.sqrt(D_squared)

    return D_squared


def kNNClassify(Xtr,Ytr,k,Xte):
    n=Xtr.shape[0]
    if k>n:
       k=n
    ylab=np.unique(Ytr)
    ym=sum(ylab)/2
    #center output if output is not in[-1,1]
    Ytrm=Ytr-ym
    I=np.argsort(distance_matrix(Xtr,Xte),axis=0)#sorting coloumn wise
    idx=I[0:k,:]
    m=Xte.shape[0]
    if k==1:
       Ypred=Ytr[idx]
    else:
        val=[sum(Ytrm[idx][:,i])/k for i in range(len(idx[0,:]))]
        for i in range(len(val)):
           if val[i]==0:
                val[i]=Ytrm[idx][0,i]
        Ypred=np.sign(val)
    return Ypred
    
#K-Nearest Neighbors¶
#Determining Optimal Number of Neighbors
#To detect optimal value for 'K', lets compare the accuracy of KNN models with different 'K' values.

#By plotting the values in a graph, we can see the performance visually, and select the best performing 'K' value


score = []

for k in range(1, 9):
  print('Begin KNN with k=',k)
  predicted=kNNClassify(X_train,train_labels,k,X_test)
  accuracy = accuracy_metric(y_valid, predicted)
  score.append(accuracy)
  print("Accuracy: ",accuracy)
  print("Confusion Matrix:\n", confusion_matrix(y_valid, pred))
  print("\n---------------\n")








#Ways to improve performance (question-5,part-(d))
#Features should be normalized to have the same range of 
#values (e.g., [0,1]), otherwise features with larger ranges will 
#have higher impact on the distance.
#Our data may also contain the GPAs
#• Should we include this attribute into the calculation?
#• When collecting data, people tend to collect as much 
#information as possible regardless whether they are 
#useful or not for the question in hand
#• It is critical to recognize and remove such attributes when 
#building your classification models 
#• Curse of dimensionality:
#– In high dimensional space (e.g., over 20), data becomes so 
#sparse that the nearest neighbor is still very far, not informative 
#any more
#Often need to be used with dimension reduction
# It can be computationally 
#expensive to find the nearest 
#neighbors for large data
#– O(Nd) for every test point (as 
#opposed to O(d) for linear 
#classifier)
#– Speed up the computation by 
#using smart data structures (kd-
#tree) to quickly search for 
#approximate solutions
#• For massive data sets, it also 
#requires a lot of memory
#– Remove unimportant examples	

# New Section

In [ ]:
pip install idx2numpy

  Created wheel for idx2numpy: filename=idx2numpy-1.2.3-cp36-none-any.whl size=7905 sha256=cdd1b3824bb0c122d6557d161ee71cb97024893fbe38be6d05b26a3d2b1341e6
  Stored in directory: /root/.cache/pip/wheels/7a/c1/da/284ce80a748fab898b8d1fa95468a386e7cf3b81da18511f9d
Successfully built idx2numpy


In [ ]:
http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz
http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz
http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz
http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz

SyntaxError: ignored

In [ ]:
pip install python-mnist

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

KeyboardInterrupt: ignored